#Data

In [ ]:

import pandas as pd
import numpy as np

##Добавил новое
!python -m spacy download en_core_web_lg
import spacy.cli
spacy.cli.download("en_core_web_lg")
import nltk
import tqdm
from nltk.corpus import stopwords
import re,string
nltk.download('all')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
import spacy
import gc
import gensim
import collections
nlp = spacy.load('en_core_web_lg')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from collections import defaultdict

2023-06-06 08:30:09.671984: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-06 08:30:10.959168: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

In [ ]:
df = pd.read_excel('all_interviews.xlsx', sheet_name=['interview1', 'interview2', 'interview3', 'interview4','interview5', 'interview6', 
                                                      'interview7', 'interview8', 'interview9', 'interview10', 'interview11', 'interview12'])
df = pd.concat(df).reset_index(drop=True)
df



,Nationality,Gender,Age,Question,Answer
0,Italian,Female,26,Thank you for agreeing to participate in this ...,I'm an Italian Master's student. I'm currently...
1,Italian,Female,26,Why have you chosen South Korea as a country f...,"Actually, I did my bachelor's in Japanese stud..."
2,Italian,Female,26,"You mentioned that you were almost, graduating...","Well, just studying or also the application?"
3,Italian,Female,26,"For example, the format of your class and stuf...","So, uh, two years ago we were in the middle of..."
4,Italian,Female,26,So South Korean university are different from ...,"Mostly, it, it's different the way, , courses ..."
...,...,...,...,...,...
379,Russian,Female,23,Did your professors give you group assignments...,I guess I was more likely to be assigned to a ...
380,Russian,Female,23,If you've had experience with both Korean and ...,"International students, again, are generally l..."
381,Russian,Female,23,How responsible are international students whe...,I think meeting deadlines depends on the perso...
382,Russian,Female,23,Who was most often late for class? What do you...,We have never had anyone be more than 1 minute...


In [ ]:
df.head(10)

,Nationality,Gender,Age,Question,Answer
0,Italian,Female,26,Thank you for agreeing to participate in this ...,I'm an Italian Master's student. I'm currently...
1,Italian,Female,26,Why have you chosen South Korea as a country f...,"Actually, I did my bachelor's in Japanese stud..."
2,Italian,Female,26,"You mentioned that you were almost, graduating...","Well, just studying or also the application?"
3,Italian,Female,26,"For example, the format of your class and stuf...","So, uh, two years ago we were in the middle of..."
4,Italian,Female,26,So South Korean university are different from ...,"Mostly, it, it's different the way, , courses ..."
5,Italian,Female,26,It is similar to the system we have here in Ru...,"Yes, it was in every class, every professor re..."
6,Italian,Female,26,It's kinda similar to the year of of online cl...,"Well, usually in class, participation is reall..."
7,Italian,Female,26,So this interaction was more about from the fo...,"Oh, yes, yes, definitely. Yeah."
8,Italian,Female,26,"Curious! moving on for another question, let's...","I've also did, my study exchange in Japan for ..."
9,Italian,Female,26,In which university?,I was at national University of Yokohama.


In [ ]:
df['Question'] = df['Question'].astype("string")
df['Answer'] = df['Answer'].astype("string")
df['Gender'] = df['Gender'].astype("string")
df['Nationality'] = df['Nationality'].astype("string")



In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Nationality  384 non-null    string
 1   Gender       384 non-null    string
 2   Age          384 non-null    int64 
 3   Question     380 non-null    string
 4   Answer       384 non-null    string
dtypes: int64(1), string(4)
memory usage: 15.1 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Nationality  7 non-null      string
 1   Gender       7 non-null      string
dtypes: string(2)
memory usage: 240.0 bytes


#Preprocessing


In [ ]:
gc.collect()
# convert to lower case, remove punctuation and numbers, replace dollar and euro signs with words. 
# remove double spaces
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text= re.sub(r'%', ' percent ', text)
    text=re.sub('\$', ' dollar ', text)
    text=re.compile('<.*?>').sub(' ', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', ' ', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    text = re.sub('\s+', ' ', text)  
    return text

# a list of uninformative short words that used to be acronyms 
# or a private name. 
not_informative_short_words =['I', 'a', 'an', 'as', 'at', 'be', 'by', 'e', 'g', 'he', 'hi', 'if', 'in', 'it', 'k', 'll', 'm', 'my', 'no', 'of', 'oh', 'on', 'or', 're', 's', 'so', 't', 'to', 'up', 've', 'we']

# removal of English particles and prepositions.
all_stopwords = stopwords.words('english')
def delete_stop_words(string):
    text_tokens = word_tokenize(string)
    tokens_without_sw = [word for word in text_tokens if not word in all_stopwords]
    filtered_sentence = (" ").join(tokens_without_sw)
    return filtered_sentence

df['Clean_sentence']=df['Answer'].apply(lambda x: preprocess(x))
df['Clean_sentence']=df.Clean_sentence.apply(delete_stop_words)
df['Clean_sentence']=df['Clean_sentence'].apply(lambda x:' '.join( [w for w in x.split() if len(w)>2]))
df['Clean_sentence']=df['Clean_sentence'].apply(lambda x:' '.join( [w for w in x.split() if w not in not_informative_short_words]))
df.head()

,Nationality,Gender,Age,Question,Answer,Clean_sentence
0,Italian,Female,26,Thank you for agreeing to participate in this ...,I'm an Italian Master's student. I'm currently...,italian master student currently studying sout...
1,Italian,Female,26,Why have you chosen South Korea as a country f...,"Actually, I did my bachelor's in Japanese stud...",actually bachelor japanese studies interested ...
2,Italian,Female,26,"You mentioned that you were almost, graduating...","Well, just studying or also the application?",well studying also application
3,Italian,Female,26,"For example, the format of your class and stuf...","So, uh, two years ago we were in the middle of...",two years ago middle pandemic universities onl...
4,Italian,Female,26,So South Korean university are different from ...,"Mostly, it, it's different the way, , courses ...",mostly different way courses organized italy c...


##NLTK

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
example=df.iloc[13,3]
print(example)
lemmatized_output = " ".join([lemmatizer.lemmatize(w) for w in example.split()])
print(lemmatized_output)

When you did this group project, would be you more willing to do it in group with Korean students, or foreigner students?
When you did this group project, would be you more willing to do it in group with Korean students, or foreigner students?


In [ ]:
df['Clean_sentence_lemm_nltk']=df['Clean_sentence'].apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in x.split()]))

In [ ]:
df.head()

,Nationality,Gender,Age,Question,Answer,Clean_sentence,Clean_sentence_lemm_nltk
0,Italian,Female,26,Thank you for agreeing to participate in this ...,I'm an Italian Master's student. I'm currently...,italian master student currently studying sout...,italian master student currently studying sout...
1,Italian,Female,26,Why have you chosen South Korea as a country f...,"Actually, I did my bachelor's in Japanese stud...",actually bachelor japanese studies interested ...,actually bachelor japanese study interested ko...
2,Italian,Female,26,"You mentioned that you were almost, graduating...","Well, just studying or also the application?",well studying also application,well studying also application
3,Italian,Female,26,"For example, the format of your class and stuf...","So, uh, two years ago we were in the middle of...",two years ago middle pandemic universities onl...,two year ago middle pandemic university online...
4,Italian,Female,26,So South Korean university are different from ...,"Mostly, it, it's different the way, , courses ...",mostly different way courses organized italy c...,mostly different way course organized italy cl...


##SPICY

In [ ]:
nlp = spacy.load('en_core_web_lg')
example=df.iloc[13,3]
print(example)
doc=nlp(example)
lemmatized_output = " ".join([token.lemma_ for token in doc])
print(lemmatized_output)

When you did this group project, would be you more willing to do it in group with Korean students, or foreigner students?
when you do this group project , would be you more willing to do it in group with korean student , or foreigner student ?


In [ ]:
df['Clean_sentence_lemm_spacy']=df['Clean_sentence'].apply(lambda x: " ".join([token.lemma_ for token in nlp(x)]))

In [ ]:
df.head()

,Nationality,Gender,Age,Question,Answer,Answer_Length,Clean_sentence,Clean_sentence_lemm_nltk,Clean_sentence_lemm_spacy
0,Italian,Female,26,Thank you for agreeing to participate in this ...,I'm an Italian Master's student. I'm currently...,3,italian master student currently studying sout...,italian master student currently studying sout...,italian master student currently study south k...
1,Italian,Female,26,Why have you chosen South Korea as a country f...,"Actually, I did my bachelor's in Japanese stud...",6,actually bachelor japanese studies interested ...,actually bachelor japanese study interested ko...,actually bachelor japanese study interest kore...
2,Italian,Female,26,"You mentioned that you were almost, graduating...","Well, just studying or also the application?",1,well studying also application,well studying also application,well study also application
3,Italian,Female,26,"For example, the format of your class and stuf...","So, uh, two years ago we were in the middle of...",9,two years ago middle pandemic universities onl...,two year ago middle pandemic university online...,two year ago middle pandemic university online...
4,Italian,Female,26,So South Korean university are different from ...,"Mostly, it, it's different the way, , courses ...",8,mostly different way courses organized italy c...,mostly different way course organized italy cl...,mostly different way course organize italy cla...


##WORDS

In [ ]:
collection1=collections.Counter([y for x in df.Clean_sentence_lemm_spacy.values.flatten() for y in x.split()])

In [ ]:
word_len={}
for x in collection1.items():
    word_len[x[0]]=len(x[0])

In [ ]:
long_words=[]
for pair in  word_len.items():
    if pair[1]>=15 and pair[1]<30:
        long_words.append(pair[0])

In [ ]:
print(long_words)

['misunderstanding', 'responsibilitie', 'straightforward', 'irresponsibility', 'interpersonally', 'professionalism', 'dissatisfaction', 'straightforwardness', 'multidimensional', 'competitiveness', 'individualistic', 'extracurricular', 'catastrophically']


In [ ]:
short_words=[]
for pair in  word_len.items():
    if pair[1]<=2:
        short_words.append(pair[0])

In [ ]:
short_words=sorted(short_words)
print(short_words)

['do', 'go']


##FINAL Preprocessing

In [ ]:
df.head()

,Nationality,Gender,Age,Question,Answer,Clean_sentence,Clean_sentence_lemm_nltk,Clean_sentence_lemm_spacy
0,Italian,Female,26,Thank you for agreeing to participate in this ...,I'm an Italian Master's student. I'm currently...,italian master student currently studying sout...,italian master student currently studying sout...,italian master student currently study south k...
1,Italian,Female,26,Why have you chosen South Korea as a country f...,"Actually, I did my bachelor's in Japanese stud...",actually bachelor japanese studies interested ...,actually bachelor japanese study interested ko...,actually bachelor japanese study interest kore...
2,Italian,Female,26,"You mentioned that you were almost, graduating...","Well, just studying or also the application?",well studying also application,well studying also application,well study also application
3,Italian,Female,26,"For example, the format of your class and stuf...","So, uh, two years ago we were in the middle of...",two years ago middle pandemic universities onl...,two year ago middle pandemic university online...,two year ago middle pandemic university online...
4,Italian,Female,26,So South Korean university are different from ...,"Mostly, it, it's different the way, , courses ...",mostly different way courses organized italy c...,mostly different way course organized italy cl...,mostly different way course organize italy cla...


In [ ]:
#df.to_excel('example.xlsx', index=False) - скачать файл и потом просто его загружать чтобы не делдать каждый раз препроцессинг (Пока оставь тебе не надо но если надо я уже скачал файл и могу скинуть тебе)

Нам не для всех блоков обязательна лемматизация
1)Оставим спайси т.к показывает себя лучше в среднем из-за удаления форм глаголов
2)В следующих блоках рассмотрим анализ наших интервью

#1st methode

ДОБРОЕ УТРО МОЯ ДОРОГАЯ, в этом методе я рассмотрел как часто люди употребляю себя или то что они делали.( как часто они это употребляли в ответе), так же рассмотрел по самим национальностям статистику как часто они это делают и так же по полу, дальше можно будет добавить по возрасту и тд. но пока так, потом напиши в лс норм думаешь или чет еще можно тут рассмотреть?


In [ ]:
# define functions to count self-referencing and action words
def count_self(text):
    return sum(1 for word in text.split() if word.lower() in ['i', 'my', 'me', 'mine', 'myself'])

def count_action(text):
    return sum(1 for word in text.split() if word.lower() in ['went', 'play', 'like', 'love', 'do', 'did', 'go', 'can'])

# apply the functions to the Answer column and count the total number of words
df['Self References'] = df['Answer'].apply(count_self)
df['Action Words'] = df['Answer'].apply(count_action)
df['Total Words'] = df['Answer'].apply(lambda x: len(x.split()))

# calculate the overall frequencies
self_freq = df['Self References'].sum() / len(df)
action_freq = df['Action Words'].sum() / len(df)

# group by nationality and gender, and calculate the frequencies and totals for each group
grouped = df.groupby(['Nationality', 'Gender']).agg(
    Self_Frequency=('Self References', lambda x: x.sum() / x.count()),
    Action_Frequency=('Action Words', lambda x: x.sum() / x.count()),
    Total_Self_References=('Self References', 'sum'),
    Total_Action_Words=('Action Words', 'sum'),
    Total_Words=('Total Words', 'sum')
)

# sort by Total_Words in descending order
grouped = grouped.sort_values(by='Total_Words', ascending=False)

# group by nationality and gender, and count the number of rows in each group
counts = df.groupby(['Nationality', 'Gender']).size().reset_index(name='Count')

# create a dictionary to store the gender counts for each nationality
gender_counts = {}
for nat in df['Nationality'].unique():
    gender_counts[nat] = counts[counts['Nationality'] == nat].sort_values(by='Count', ascending=False)

# print the results for each nationality
for nat in gender_counts:
    print(f'Nationality: {nat}')
    for idx, row in gender_counts[nat].iterrows():
        gender = row['Gender']
        count = row['Count']
        print(f'{gender}: {count} content items')
    print()

# print the overall results
print(grouped)




Nationality: Italian
Female: 23 content items
Male: 16 content items

Nationality: The Netherland
Male: 58 content items

Nationality: Chinese
Male: 48 content items
Female: 40 content items

Nationality: Russian
Male: 136 content items
Female: 63 content items

                       Self_Frequency  Action_Frequency  \
Nationality    Gender                                     
Chinese        Female        6.850000          5.100000   
Russian        Male          1.735294          0.610294   
               Female        2.555556          1.888889   
Chinese        Male          3.291667          1.187500   
The Netherland Male          2.189655          2.172414   
Italian        Female        3.000000          2.086957   
               Male          4.562500          2.000000   

                       Total_Self_References  Total_Action_Words  Total_Words  
Nationality    Gender                                                          
Chinese        Female                    274 

In [ ]:
df

,Nationality,Gender,Age,Question,Answer,Clean_sentence,Clean_sentence_lemm_nltk,Clean_sentence_lemm_spacy,Self References,Action Words,Total Words
0,Italian,Female,26,Thank you for agreeing to participate in this ...,I'm an Italian Master's student. I'm currently...,italian master student currently studying sout...,italian master student currently studying sout...,italian master student currently study south k...,0,0,17
1,Italian,Female,26,Why have you chosen South Korea as a country f...,"Actually, I did my bachelor's in Japanese stud...",actually bachelor japanese studies interested ...,actually bachelor japanese study interested ko...,actually bachelor japanese study interest kore...,5,1,68
2,Italian,Female,26,"You mentioned that you were almost, graduating...","Well, just studying or also the application?",well studying also application,well studying also application,well study also application,0,0,7
3,Italian,Female,26,"For example, the format of your class and stuf...","So, uh, two years ago we were in the middle of...",two years ago middle pandemic universities onl...,two year ago middle pandemic university online...,two year ago middle pandemic university online...,7,3,198
4,Italian,Female,26,So South Korean university are different from ...,"Mostly, it, it's different the way, , courses ...",mostly different way courses organized italy c...,mostly different way course organized italy cl...,mostly different way course organize italy cla...,2,2,149
...,...,...,...,...,...,...,...,...,...,...,...
379,Russian,Female,23,Did your professors give you group assignments...,I guess I was more likely to be assigned to a ...,guess likely assigned group foreigners cases k...,guess likely assigned group foreigner case kor...,guess likely assign group foreigner case korea...,3,1,43
380,Russian,Female,23,If you've had experience with both Korean and ...,"International students, again, are generally l...",international students generally like assignme...,international student generally like assignmen...,international student generally like assignmen...,2,2,94
381,Russian,Female,23,How responsible are international students whe...,I think meeting deadlines depends on the perso...,think meeting deadlines depends person respons...,think meeting deadline depends person responsi...,think meeting deadline depend person responsib...,1,0,22
382,Russian,Female,23,Who was most often late for class? What do you...,We have never had anyone be more than 1 minute...,never anyone minute late class attendance coun...,never anyone minute late class attendance coun...,never anyone minute late class attendance coun...,0,0,31


# ACTIVITY - REFLEXIVITY
# (if a person uses shorter sentences - active, and longer sentences - reflective)

In [ ]:

df['Answer_Length'] = df['Answer'].apply(lambda x: len(str(x).split('.')))
df['Words_in_sent']= df['Total Words']/df['Answer_Length']

grouped_data = df.groupby(['Nationality', 'Gender'])['Words_in_sent'].agg(['mean', 'max', 'min'])

print(grouped_data)



                            mean        max       min
Nationality    Gender                                
Chinese        Female  14.413469  25.833333  4.500000
               Male    11.577906  26.666667  1.000000
Italian        Female  12.699258  22.857143  1.666667
               Male    10.646345  14.250000  2.500000
Russian        Female  12.779999  22.000000  1.500000
               Male     9.087103  38.000000  0.500000
The Netherland Male    11.634743  32.800000  1.000000


# SENTIMENTAL ANALYSIS



In [ ]:
analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):

    scores = analyzer.polarity_scores(text)["compound"]
    return scores

df['sentiment'] = df['Clean_sentence_lemm_nltk'].apply(get_sentiment)

def sent_to_text(num):
    if num < -0.8:
        return 'strongly negative' 

    elif num < 0 and num > -0.4:
        return 'negative'

    elif num > -0.4 and num < 0.2:
        return 'neutral' 

    elif num < 0.6 and num > 0.2:
        return 'positive' 

    else :
        return 'very positive'

df['sent_to_text'] = df['sentiment'].apply(sent_to_text)

df.head(10)


,Nationality,Gender,Age,Question,Answer,Clean_sentence,Clean_sentence_lemm_nltk,Clean_sentence_lemm_spacy,Self References,Action Words,Total Words,Answer_Length,Words_in_sent,Negative words,sentiment,sent_to_text
0,Italian,Female,26,Thank you for agreeing to participate in this ...,I'm an Italian Master's student. I'm currently...,italian master student currently studying sout...,italian master student currently studying sout...,italian master student currently study south k...,0,0,17,3,5.666667,0,0.0000,neutral
1,Italian,Female,26,Why have you chosen South Korea as a country f...,"Actually, I did my bachelor's in Japanese stud...",actually bachelor japanese studies interested ...,actually bachelor japanese study interested ko...,actually bachelor japanese study interest kore...,5,1,68,6,11.333333,0,0.8176,very positive
2,Italian,Female,26,"You mentioned that you were almost, graduating...","Well, just studying or also the application?",well studying also application,well studying also application,well study also application,0,0,7,1,7.000000,0,0.2732,positive
3,Italian,Female,26,"For example, the format of your class and stuf...","So, uh, two years ago we were in the middle of...",two years ago middle pandemic universities onl...,two year ago middle pandemic university online...,two year ago middle pandemic university online...,7,3,198,9,22.000000,1,0.8316,very positive
4,Italian,Female,26,So South Korean university are different from ...,"Mostly, it, it's different the way, , courses ...",mostly different way courses organized italy c...,mostly different way course organized italy cl...,mostly different way course organize italy cla...,2,2,149,8,18.625000,2,0.6369,very positive
5,Italian,Female,26,It is similar to the system we have here in Ru...,"Yes, it was in every class, every professor re...",yes every class every professor required keep ...,yes every class every professor required keep ...,yes every class every professor require keep t...,3,2,97,5,19.400000,0,0.7096,very positive
6,Italian,Female,26,It's kinda similar to the year of of online cl...,"Well, usually in class, participation is reall...",well usually class participation really encour...,well usually class participation really encour...,well usually class participation really encour...,4,5,132,11,12.000000,2,0.9284,very positive
7,Italian,Female,26,So this interaction was more about from the fo...,"Oh, yes, yes, definitely. Yeah.",yes yes definitely yeah,yes yes definitely yeah,yes yes definitely yeah,0,0,5,3,1.666667,0,0.8519,very positive
8,Italian,Female,26,"Curious! moving on for another question, let's...","I've also did, my study exchange in Japan for ...",also study exchange japan one semester four ye...,also study exchange japan one semester four ye...,also study exchange japan one semester four ye...,1,0,14,2,7.000000,0,0.0000,neutral
9,Italian,Female,26,In which university?,I was at national University of Yokohama.,national university yokohama,national university yokohama,national university yokohama,1,0,7,2,3.500000,0,0.0000,neutral


In [ ]:
pd.crosstab(df['Nationality'],  df['sent_to_text'], margins=True)

sent_to_text,negative,neutral,positive,strongly negative,very positive,All
Nationality,,,,,,
Chinese,4,4,14,2,64,88
Italian,0,4,3,0,32,39
Russian,11,41,44,2,101,199
The Netherland,1,9,18,0,30,58
All,16,58,79,4,227,384


# STRIVING - AVOIDING
# (How often do you hear negative language (with the particle "not"))

In [ ]:
def count_negative(text):
    return sum(1 for word in text.split() if word.lower() in ['not', "don't", "didn't", "doesn't", "won't", "no", "never"])

df['Negative words']= df['Answer'].apply(count_negative)
num_negative = {
    'Negative words': ['min', 'max']
}
table = df.groupby(['Nationality', 'Gender']).agg(num_negative)

grouped_table = df.groupby(['Nationality', 'Gender']).agg(
    Negative_Frequency=('Negative words', lambda x: x.sum() / x.count()))

print(table)
print(grouped_table)
















                      Negative words    
                                 min max
Nationality    Gender                   
Chinese        Female              0   8
               Male                0   6
Italian        Female              0   4
               Male                0   7
Russian        Female              0   8
               Male                0   6
The Netherland Male                0  12
                       Negative_Frequency
Nationality    Gender                    
Chinese        Female            2.425000
               Male              1.229167
Italian        Female            0.956522
               Male              1.812500
Russian        Female            1.412698
               Male              1.014706
The Netherland Male              1.086207


# PARTS OF SPEACH

In [ ]:

pos_counts = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))

for index, row in df.iterrows():

    answer = row['Answer']
    gender = row['Gender']
    nationality = row['Nationality']

    sentences = nltk.sent_tokenize(answer)
    for sentence in sentences:
        words = word_tokenize(sentence)
        tagged_words = pos_tag(words)
        for word, pos in tagged_words:
            if pos.startswith('N'):
                pos_counts[nationality][gender]['Noun'] += 1
            elif pos.startswith('V'):
                pos_counts[nationality][gender]['Verb'] += 1
            elif pos.startswith('J'):
                pos_counts[nationality][gender]['Adjective'] += 1
            elif pos.startswith('P'):
                pos_counts[nationality][gender]['Pronoun'] += 1
            elif pos.startswith('R'):
                pos_counts[nationality][gender]['Adverb'] += 1


for nationality, gender_counts in pos_counts.items():
    print(f"Nationality: {nationality}")
    for gender, counts in gender_counts.items():
        print(f"Gender: {gender}")
        total = sum(counts.values())
        for pos, count in counts.items():
            percentage = (count / total)* 100 
            print(f"{pos}: {count} ({percentage:.2f}%)")
        print()


Nationality: Italian
Gender: Female
Pronoun: 249 (18.89%)
Verb: 331 (25.11%)
Adjective: 150 (11.38%)
Noun: 335 (25.42%)
Adverb: 253 (19.20%)

Gender: Male
Pronoun: 277 (20.87%)
Noun: 309 (23.29%)
Verb: 405 (30.52%)
Adjective: 159 (11.98%)
Adverb: 177 (13.34%)

Nationality: The Netherland
Gender: Male
Pronoun: 343 (18.38%)
Verb: 473 (25.35%)
Noun: 478 (25.62%)
Adjective: 206 (11.04%)
Adverb: 366 (19.61%)

Nationality: Chinese
Gender: Male
Noun: 677 (27.80%)
Pronoun: 448 (18.40%)
Verb: 670 (27.52%)
Adjective: 330 (13.55%)
Adverb: 310 (12.73%)

Gender: Female
Pronoun: 1142 (18.65%)
Verb: 1693 (27.65%)
Adverb: 807 (13.18%)
Adjective: 879 (14.36%)
Noun: 1601 (26.15%)

Nationality: Russian
Gender: Male
Pronoun: 760 (16.93%)
Verb: 1248 (27.81%)
Adjective: 503 (11.21%)
Noun: 1315 (29.30%)
Adverb: 662 (14.75%)

Gender: Female
Pronoun: 773 (17.77%)
Verb: 1202 (27.63%)
Noun: 1335 (30.69%)
Adjective: 512 (11.77%)
Adverb: 528 (12.14%)

